In [1]:
print("hello")

hello


# **Enhance LLMs using RAG and Hugging Face**


Imagine you've been hired to help the HR department build an intelligent question-answering tool for company policies. Employees can input questions such as "What is our vacation policy?" or "How do I submit a reimbursement request?" and receive instant, clear answers. This tool would save time and help employees understand complex policy documents easily, by automatically providing relevant information instead of searching through pages of text.


In this lab, you'll delve into the advanced concept of Retriever-Augmented Generation (RAG), a cutting-edge approach in natural language processing that synergistically combines the powers of retrieval and generation. You will explore how to effectively retrieve relevant information from a large dataset and then use a state-of-the-art sequence-to-sequence model to generate precise answers to complex questions. By integrating tools such as the Dense Passage Retriever (DPR) and the GPT2 model for generation, this lab will equip you with the skills to build a sophisticated question-answering system that can find and synthesize information on-the-fly. Through hands-on coding exercises and implementations, you will gain practical experience in handling real-world NLP challenges, setting up a robust natural language processing (NLP) pipeline, and fine-tuning models to enhance their accuracy and relevance.


## Objectives


- **Understand the concept and components:** Grasp the fundamentals of Retriever-Augmented Generation (RAG), focusing on how retrieval and generation techniques are combined in natural language processing (NLP).
- **Implement Dense Passage Retriever (DPR):** Learn to set up and use DPR to efficiently retrieve documents from a large dataset, which is crucial for feeding relevant information into generative models.
- **Integrate sequence-to-sequence models:** Explore integrating sequence-to-sequence models such as GPT2 to generate answers based on the contexts provided by DPR, enhancing the accuracy and relevance of responses.
- **Build a Question-Answering System:** Gain practical experience by developing a question-answering system that utilizes both DPR and GPT2, mimicking real-world applications.
- **Fine-tune and optimize NLP models:** Acquire skills in fine-tuning and optimizing NLP models to improve their performance and suitability for specific tasks or datasets.
- **Use professional NLP tools:** Get familiar with using advanced NLP tools and libraries, such as Hugging Face’s transformers and dataset libraries, to implement sophisticated NLP solutions.


In this lab, you'll use several libraries tailored for natural language processing, data manipulation, and efficient computation:

- **[wget](https://pypi.org/project/wget/)**: Used to download files from the internet, essential for fetching datasets or pretrained models.

- **[torch](https://pytorch.org/)**: PyTorch library, fundamental for machine learning and neural network operations, provides GPU acceleration and dynamic neural network capabilities.

- **[numpy](https://numpy.org/)**: A staple for numerical operations in Python, used for handling arrays and matrices.

- **[faiss](https://github.com/facebookresearch/faiss)**: Specialized for efficient similarity search and clustering of dense vectors, crucial for information retrieval tasks.

- **[transformers](https://huggingface.co/transformers/)**: Offers a multitude of pretrained models for a variety of NLP tasks, for example:
  
  **DPRQuestionEncoder**, **DPRContextEncoder**: Encode questions and contexts into vector embeddings for retrieval.

- **[tokenizers](https://huggingface.co/docs/tokenizers/)**: Tools that convert input text into numerical representations (tokens) compatible with specific models, ensuring effective processing and understanding by the models, for example:

  **[DPRQuestionEncoderTokenizer](https://huggingface.co/transformers/model_doc/dpr.html)**, **[DPRContextEncoderTokenizer](https://huggingface.co/transformers/model_doc/dpr.html)**: Convert text into formats suitable for their respective models, ensuring optimal performance for processing and generating text.

These tools are integral to developing the question-answering system in this lab, covering everything from data downloading and preprocessing to advanced machine learning tasks.


In [3]:
pip install wget

  Preparing metadata (setup.py) ... done
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9655 sha256=8ef648a7c58c1c28524cedf54205801d2e632c4c7a4431c75ae6ff5ee5a9e25a
  Stored in directory: /root/.cache/pip/wheels/40/b3/0f/a40dbd1c6861731779f62cc4babcb234387e11d697df70ee97
Successfully built wget


In [4]:
!pip install --user transformers datasets torch faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.4/491.4 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 76.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 63.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 43.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 57.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 39.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [5]:
import wget
from transformers import DPRContextEncoder, DPRContextEncoderTokenizer
import torch

import numpy as np
import random
from transformers import DPRQuestionEncoder, DPRQuestionEncoderTokenizer
from transformers import AutoTokenizer, AutoModelForCausalLM


import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from sklearn.manifold import TSNE
import numpy as np

# You can also use this section to suppress warnings generated by your code:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn
warnings.filterwarnings('ignore')